### Download the Data

https://www.w3resource.com/sql-exercises/movie-database-exercise/joins-exercises-on-movie-database.php

In [1]:
!pip install wget

  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9681 sha256=6236d65ec2fa1a7724e4ae55ebc5fadd75c22b0b95a7b160120311510b7e2828
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


In [2]:
import wget
fn = wget.download('https://github.com/RJ2494/from_sql_to_pandas/raw/master/movie_rating.zip')
fn

'movie_rating.zip'

In [3]:
# !unzip $fn
!unzip movie_rating.zip

Archive:  movie_rating.zip
   creating: movie_rating/
  inflating: movie_rating/orders.csv  
  inflating: movie_rating/actor.csv  
  inflating: movie_rating/customers.csv  
  inflating: movie_rating/director.csv  
  inflating: movie_rating/genre.csv  
  inflating: movie_rating/genres.csv  
  inflating: movie_rating/movie.csv  
  inflating: movie_rating/movie_cast.csv  
  inflating: movie_rating/movie_direction.csv  
  inflating: movie_rating/movie_genres.csv  
  inflating: movie_rating/rating.csv  
  inflating: movie_rating/reviewer.csv  


### Basic Info About Data

In [0]:
PATH = './movie_rating'
# PATH = '.'

In [5]:
!find $PATH -name '*.csv' | xargs wc -l | sort -nr

 221 total
  29 ./movie_rating/movie.csv
  25 ./movie_rating/actor.csv
  24 ./movie_rating/movie_direction.csv
  24 ./movie_rating/movie_cast.csv
  24 ./movie_rating/director.csv
  21 ./movie_rating/reviewer.csv
  20 ./movie_rating/rating.csv
  20 ./movie_rating/movie_genres.csv
  14 ./movie_rating/genres.csv
  14 ./movie_rating/genre.csv
   3 ./movie_rating/orders.csv
   3 ./movie_rating/customers.csv


In [6]:
# find which seperator is used to seperate the columns of each csv file
!head -1 $PATH/*.csv

==> ./movie_rating/actor.csv <==
act_id|act_fname|act_lname|act_gender

==> ./movie_rating/customers.csv <==
CustomerID|CustomerName|ContactName|Address|City|PostalCode|Country

==> ./movie_rating/director.csv <==
dir_id|dir_fname|dir_lname

==> ./movie_rating/genre.csv <==
gen_id|gen_title

==> ./movie_rating/genres.csv <==
gen_id|gen_title

==> ./movie_rating/movie_cast.csv <==
act_id|mov_id|role

==> ./movie_rating/movie.csv <==
mov_id|mov_title|mov_year|mov_time|mov_lang|mov_dt_rel|mov_rel_country

==> ./movie_rating/movie_direction.csv <==
dir_id|mov_id

==> ./movie_rating/movie_genres.csv <==
mov_id|gen_id

==> ./movie_rating/orders.csv <==
OrderID|CustomerID|EmployeeID|OrderDate|ShipperID

==> ./movie_rating/rating.csv <==
mov_id|rev_id|rev_stars|num_of_ratings

==> ./movie_rating/reviewer.csv <==
rev_id|rev_name


> * We can see that all files are `|`  seperated.

### Read the Dataset

In [0]:
import pandas as pd
import numpy as np
import re
import os

from collections import defaultdict

In [0]:
pd.set_option('display.width', 1000)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_colwidth', 1000)

In [0]:
db = defaultdict(pd.DataFrame)

for filename in os.listdir(PATH):
    if filename.endswith('.csv'):
        db[filename.split('.')[0]] = pd.read_csv(f"{PATH}/{filename}", sep='|')
        

In [10]:
pd.DataFrame(db.items(), columns=['table_name', 'table_data'])

,table_name,table_data
0,movie_cast,act_id mov_id role 0 101 901 JohnScottieFerguson 1 102 902 MissGiddens 2 103 903 T.E.Lawrence 3 104 904 Michael 4 105 905 AntonioSalieri 5 106 906 RickDeckard 6 107 907 AliceHarford 7 108 908 McManus 8 110 910 EddieAdams 9 111 911 AlvySinger 10 112 912 San 11 113 913 AndyDufresne 12 114 914 LesterBurnham 13 115 915 RoseDeWittBukater 14 116 916 SeanMaguire 15 117 917 Ed 16 118 918 Renton 17 120 920 ElizabethDarko 18 121 921 OlderJamal 19 122 922 Ripley 20 114 923 BobbyDarin 21 109 909 J.J.Gittes 22 119 919 AlfredBorden
1,customers,CustomerID CustomerName ContactName Address City PostalCode Country 0 1 Alfreds Futterkiste Maria Anders Obere Str. 57 Berlin 12209 Germany 1 2 Ana Trujillo Emparedados y helados Ana Trujillo Avda. de la Constitución 2222 México D.F. 5021 Mexico 2 3 Antonio Moreno Taquería Antonio Moreno Mataderos 2312 México D.F. 5023 Mexico
2,actor,act_id act_fname act_lname act_gender 0 101 James Stewart M 1 102 Deborah Kerr F 2 103 Peter OToole M 3 104 Robert DeNiro M 4 105 F.Murray Abraham M 5 106 Harrison Ford M 6 107 Nicole Kidman F 7 108 Stephen Baldwin M 8 109 Jack Nicholson M 9 110 Mark Wahlberg M 10 111 Woody Allen M 11 112 Claire Danes F 12 113 Tim Robbins M 13 114 Kevin Spacey M 14 115 Kate Winslet F 15 116 Robin Williams M 16 117 Jon Voight M 17 118 Ewan McGregor M 18 119 Christian Bale M 19 120 Maggie Gyllenhaal F 20 121 Dev Patel M 21 ...
3,orders,OrderID CustomerID EmployeeID OrderDate ShipperID 0 10308 2 7 1996-09-18 3 1 10309 37 3 1996-09-19 1 2 10310 77 8 1996-09-20 2
4,director,dir_id dir_fname dir_lname 0 201 Alfred Hitchcock 1 202 Jack Clayton 2 203 David Lean 3 204 Michael Cimino 4 205 Milos Forman 5 206 Ridley Scott 6 207 Stanley Kubrick 7 208 Bryan Singer 8 209 Roman Polanski 9 210 Paul ThomasAnderson 10 211 Woody Allen 11 212 Hayao Miyazaki 12 213 Frank Darabont 13 214 Sam Mendes 14 215 James Cameron 15 216 Gus VanSant 16 217 John Boorman 17 218 Danny Boyle 18 219 Christopher Nolan 19 220 Richard Kelly 20 221 Kevin Spacey 21 222 Andrei Tarkovsky 22 223 Peter Jackson
5,movie,mov_id mov_title mov_year mov_time mov_lang mov_dt_rel mov_rel_country 0 901 Vertigo 1958 128 English 1958-08-24 UK 1 902 TheInnocents 1961 100 English 1962-02-19 SW 2 903 LawrenceofArabia 1962 216 English 1962-12-11 UK 3 904 TheDeerHunter 1978 183 English 1979-03-08 UK 4 905 Amadeus 1984 160 English 1985-01-07 UK 5 906 BladeRunner 1982 117 English 1982-09-09 UK 6 907 EyesWideShut 1999 159 English NaN UK 7 908 TheUsualSuspects 1995 106 English 1995-08-25 UK 8 909 Chinatown 1974 130 English 1974-08-09 UK 9 910 BoogieNights 1997 155 English ...
6,genres,gen_id gen_title 0 1001 Action 1 1002 Adventure 2 1003 Animation 3 1004 Biography 4 1005 Comedy 5 1006 Crime 6 1007 Drama 7 1008 Horror 8 1009 Music 9 1010 Mystery 10 1011 Romance 11 1012 Thriller 12 1013 War
7,reviewer,rev_id rev_name 0 9001 RightySock 1 9002 JackMalvern 2 9003 FlagrantBaronessa 3 9004 AlecShaw 4 9005 NaN 5 9006 VictorWoeltjen 6 9007 SimonWright 7 9008 NealWruck 8 9009 PaulMonks 9 9010 MikeSalvati 10 9011 NaN 11 9012 WesleyS.Walker 12 9013 SashaGoldshtein 13 9014 JoshCates 14 9015 KrugStillo 15 9016 ScottLeBrun 16 9017 HannahSteele 17 9018 VincentCadena 18 9019 BrandtSponseller 19 9020 RichardAdams
8,genre,gen_id gen_title 0 1001 Action 1 1002 Adventure 2 1003 Animation 3 1004 Biography 4 1005 Comedy 5 1006 Crime 6 1007 Drama 7 1008 Horror 8 1009 Music 9 1010 Mystery 10 1011 Romance 11 1012 Thriller 12 1013 War
9,movie_genres,mov_id gen_id 0 922 1001 1 917 1002 2 903 1002 3 912 1003 4 911 1005 5 908 1006 6 913 1006 7 926 1007 8 928 1007 9 918 1007 10 921 1007 11 902 1008 12 923 1009 13 907 1010 14 927 1010 15 901 1010 16 914 1011 17 906 1012 18 904 1013


### Connect To Database Engine - 
> * We use sqlite database engine to perform sql query
> * There are many packages to connect to sqlite engine such as - sqlite3, sqlalchemy etc.
> * We use sqlalchemy to connect to database engine. It works with many Relational Database Management Systems.
> * We will use pandas DataFrame class to represent the result of a sql query into a table form.

In [0]:
from sqlalchemy import create_engine

In [0]:
engine = create_engine("sqlite:///movie_rating.sqlite")

In [13]:
# print the names of table in the database movie_review.sqlite
table_names = engine.table_names()
pd.DataFrame([table_names])

""
0


> * Since there is no table in movie_review.sqlite database, we will create some tables and insert data of movie review into them.

> * https://sqlite.org/cli.html

In [0]:
 # connect to database
con = engine.connect()

# create table one by one 
for table_name, table_data in db.items():    
    table_data.to_sql(table_name, con=engine, index=False,
                      if_exists='replace')        

In [15]:
# print the names of table in the database movie_review.sqlite
table_names = engine.table_names()
pd.DataFrame([table_names])

,0,1,2,3,4,5,6,7,8,9,10,11
0,actor,customers,director,genre,genres,movie,movie_cast,movie_direction,movie_genres,orders,rating,reviewer


### Query 26: *`INNER JOIN`* - Display role of casts for each movie.

> 


![INNER JOIN](https://www.w3resource.com/w3r_images/sql-inner-jon.gif)

> * **SQL** 

In [0]:
table_name1, table_name2 = 'movie', 'rating'

In [17]:
rs = con.execute(f'''SELECT {table_name1}.mov_title, {table_name2}.rev_stars
                     FROM {table_name1}  AS {table_name1}
                     INNER JOIN {table_name2} AS {table_name2}
                     ON {table_name1}.mov_id = {table_name2}.mov_id
                     ''')
pd.DataFrame(rs.fetchall(), columns=['movie_title', 'role']).T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
movie_title,Vertigo,TheInnocents,LawrenceofArabia,BladeRunner,TheUsualSuspects,Chinatown,BoogieNights,AnnieHall,PrincessMononoke,AmericanBeauty,Titanic,GoodWillHunting,Trainspotting,DonnieDarko,SlumdogMillionaire,Aliens,BeyondtheSea,Avatar,Braveheart
role,8.4,7.9,8.3,8.2,8.6,NaN,3,8.1,8.4,7,7.7,4,NaN,8.1,8,8.4,6.7,7.3,7.7


> * **Pandas** 

In [18]:
table1, table2 = db[table_name1], db[table_name2]

JOIN = pd.merge
INNER, ON = 'inner', 'mov_id'

FROM_CLAUSE_WITH_JOIN = JOIN(table1, table2, INNER, ON)

SELECT_CLAUSE = ['mov_title', 'rev_stars']

# pd.merge(left=db[table_name1], right=db[table_name2], how='inner') \
#                                    [['mov_title', 'rev_stars']]
res = FROM_CLAUSE_WITH_JOIN[SELECT_CLAUSE]
res.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
mov_title,Vertigo,TheInnocents,LawrenceofArabia,BladeRunner,TheUsualSuspects,Chinatown,BoogieNights,AnnieHall,PrincessMononoke,AmericanBeauty,Titanic,GoodWillHunting,Trainspotting,DonnieDarko,SlumdogMillionaire,Aliens,BeyondtheSea,Avatar,Braveheart
rev_stars,8.4,7.9,8.3,8.2,8.6,NaN,3,8.1,8.4,7,7.7,4,NaN,8.1,8,8.4,6.7,7.3,7.7


### Query 27: *`LEFT OUTER JOIN`* - Display the movie title and their review stars. Also display the movie for which we don't have review stars.
![LEFT OUTER JOIN](https://www.w3resource.com/w3r_images/sql-left-jon.png)

> * **SQL** 

In [0]:
table_name1 = 'movie'
table_name2 = 'rating'

In [20]:
rs = con.execute(f'''SELECT {table_name1}.mov_title, {table_name2}.rev_stars
                     FROM {table_name1}  AS {table_name1}
                     LEFT JOIN {table_name2} AS {table_name2}
                     ON {table_name1}.mov_id = {table_name2}.mov_id
                     ''')
pd.DataFrame(rs.fetchall(), columns=['movie_title', 'rev_stars']).T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27
movie_title,Vertigo,TheInnocents,LawrenceofArabia,TheDeerHunter,Amadeus,BladeRunner,EyesWideShut,TheUsualSuspects,Chinatown,BoogieNights,AnnieHall,PrincessMononoke,TheShawshankRedemption,AmericanBeauty,Titanic,GoodWillHunting,Deliverance,Trainspotting,ThePrestige,DonnieDarko,SlumdogMillionaire,Aliens,BeyondtheSea,Avatar,SevenSamurai,SpiritedAway,BacktotheFuture,Braveheart
rev_stars,8.4,7.9,8.3,NaN,NaN,8.2,NaN,8.6,NaN,3,8.1,8.4,NaN,7,7.7,4,NaN,NaN,NaN,8.1,8,8.4,6.7,7.3,NaN,NaN,NaN,7.7


> * **Pandas** 

In [21]:
table1, table2 = db[table_name1], db[table_name2]
JOIN = pd.merge
LEFT_OUTER, ON = 'left', 'mov_id'
FROM_CLAUSE_WITH_JOIN = JOIN(table1, table2, LEFT_OUTER, ON)

SELECT_CLAUSE = ['mov_title', 'rev_stars']

# pd.merge(left=db[table_name1], right=db[table_name2], how='inner') \
#                                    [['mov_title', 'role']]
res = FROM_CLAUSE_WITH_JOIN[SELECT_CLAUSE]
res.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27
mov_title,Vertigo,TheInnocents,LawrenceofArabia,TheDeerHunter,Amadeus,BladeRunner,EyesWideShut,TheUsualSuspects,Chinatown,BoogieNights,AnnieHall,PrincessMononoke,TheShawshankRedemption,AmericanBeauty,Titanic,GoodWillHunting,Deliverance,Trainspotting,ThePrestige,DonnieDarko,SlumdogMillionaire,Aliens,BeyondtheSea,Avatar,SevenSamurai,SpiritedAway,BacktotheFuture,Braveheart
rev_stars,8.4,7.9,8.3,NaN,NaN,8.2,NaN,8.6,NaN,3,8.1,8.4,NaN,7,7.7,4,NaN,NaN,NaN,8.1,8,8.4,6.7,7.3,NaN,NaN,NaN,7.7


### Query 28: *`RIGHT OUTER JOIN`* - Find out the movie id and, its director id and director's first name.
![**RIGHT OUTER JOIN**](https://www.w3resource.com/w3r_images/sql-right-jon.gif)

> * **SQL** - Since SQLite does not support RIGHT OUTER and FULL OUTER JOIN. We will use LEFT OUTER JOIN and combination of LEFT OUTER and UNION to perform RIGHT OUTER and FULL OUTER JOIN respectively.

In [0]:
table_name1, table_name2 = 'movie_direction', 'director'

In [23]:
rs = con.execute(f'''SELECT {table_name1}.mov_id, {table_name2}.dir_id, {table_name2}.dir_fname
                     FROM {table_name2} AS {table_name2}
                     LEFT JOIN  {table_name1}  AS {table_name1}
                     ON {table_name1}.dir_id = {table_name2}.dir_id
                     ''')
pd.DataFrame(rs.fetchall(), columns=['mov_id', 'dir_id', 'dir_name']).T

# rs = con.execute(f'''SELECT {table_name1}.mov_id, {table_name2}.dir_id, {table_name2}.dir_fname
#                      FROM {table_name1}  AS {table_name1}
#                      RIGHT JOIN  {table_name2} AS {table_name2}
#                      ON {table_name1}.dir_id = {table_name2}.dir_id
#                      ''')

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24
mov_id,901,902,903,904,905,906,907,908,909,910,911,912,913,914,915,922,916,917,918,921,919,920,923,NaN,NaN
dir_id,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,215,216,217,218,218,219,220,221,222,223
dir_name,Alfred,Jack,David,Michael,Milos,Ridley,Stanley,Bryan,Roman,Paul,Woody,Hayao,Frank,Sam,James,James,Gus,John,Danny,Danny,Christopher,Richard,Kevin,Andrei,Peter


> * **Pandas** 

In [24]:
table1 = db[table_name1]
table2 = db[table_name2]

JOIN = pd.merge
RIGHT_OUTER = 'right'    
ON = 'dir_id'
FROM_CLAUSE_WITH_JOIN = JOIN(table1, table2, RIGHT_OUTER, ON)

SELECT_CLAUSE = ['mov_id', 'dir_id', 'dir_fname']

# pd.merge(left=db[table_name1], right=db[table_name2], how='inner') \
#                                    [['mov_title', 'role']]
res = FROM_CLAUSE_WITH_JOIN[SELECT_CLAUSE]
res.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24
mov_id,901,902,903,904,905,906,907,908,909,910,911,912,913,914,915,922,916,917,918,921,919,920,923,NaN,NaN
dir_id,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,215,216,217,218,218,219,220,221,222,223
dir_fname,Alfred,Jack,David,Michael,Milos,Ridley,Stanley,Bryan,Roman,Paul,Woody,Hayao,Frank,Sam,James,James,Gus,John,Danny,Danny,Christopher,Richard,Kevin,Andrei,Peter


### Query 29: *`FULL OUTER JOIN`* - Perfrom full outer join between customers and orders
![**SQL FULL OUTER JOIN**](https://www.w3resource.com/w3r_images/sql-full-outer-join.gif)

> * **SQL** - Since SQLite does not support RIGHT OUTER and FULL OUTER JOIN. We will use LEFT OUTER JOIN and combination of LEFT OUTER and UNION to perform RIGHT OUTER and FULL OUTER JOIN respectively.

In [0]:
table_name1 = 'customers'
table_name2 = 'orders'

In [26]:
rs = con.execute(f'''SELECT  {table_name1}.CustomerID, CustomerName, ContactName, Address, City, PostalCode, Country, 
                                OrderID,	EmployeeID, OrderDate, ShipperID
                     FROM {table_name1}  AS {table_name1}
                     LEFT JOIN {table_name2} AS {table_name2}
                     ON {table_name1}.CustomerID = {table_name2}.CustomerID
                     UNION
                     SELECT {table_name2}.CustomerID, CustomerName, ContactName, Address, City, PostalCode, 
                                Country, OrderID,	EmployeeID, OrderDate, ShipperID
                     FROM {table_name2}  AS {table_name2}
                     LEFT JOIN {table_name1} AS {table_name1}
                     ON {table_name2}.CustomerID = {table_name1}.CustomerID
                     
                     ''')
pd.DataFrame(rs.fetchall(), columns = ['CustomerID', 'CustomerName', 'ContactName', 
                                       'Address', 'City', 'PostalCode', 'Country', 
                                       'OrderID', 'EmployeeID', 'OrderDate', 'ShipperID'])

,CustomerID,CustomerName,ContactName,Address,City,PostalCode,Country,OrderID,EmployeeID,OrderDate,ShipperID
0,1,Alfreds Futterkiste,Maria Anders,Obere Str. 57,Berlin,12209.0,Germany,NaN,NaN,None,NaN
1,2,Ana Trujillo Emparedados y helados,Ana Trujillo,Avda. de la Constitución 2222,México D.F.,5021.0,Mexico,10308.0,7.0,1996-09-18,3.0
2,3,Antonio Moreno Taquería,Antonio Moreno,Mataderos 2312,México D.F.,5023.0,Mexico,NaN,NaN,None,NaN
3,37,None,None,None,None,NaN,None,10309.0,3.0,1996-09-19,1.0
4,77,None,None,None,None,NaN,None,10310.0,8.0,1996-09-20,2.0


> * **Pandas** 

In [27]:
table1 = db[table_name1]
table2 = db[table_name2]

JOIN = pd.merge
FULL_OUTER = 'outer'    
ON = 'CustomerID'
FROM_CLAUSE_WITH_JOIN = JOIN(table1, table2, FULL_OUTER, ON)

SELECT_CLAUSE = ['CustomerID', 'CustomerName', 'ContactName', 'Address', 'City', 'PostalCode', 'Country', 'OrderID', 'EmployeeID', 'OrderDate', 'ShipperID']

res = FROM_CLAUSE_WITH_JOIN[SELECT_CLAUSE]
res

,CustomerID,CustomerName,ContactName,Address,City,PostalCode,Country,OrderID,EmployeeID,OrderDate,ShipperID
0,1,Alfreds Futterkiste,Maria Anders,Obere Str. 57,Berlin,12209.0,Germany,NaN,NaN,NaN,NaN
1,2,Ana Trujillo Emparedados y helados,Ana Trujillo,Avda. de la Constitución 2222,México D.F.,5021.0,Mexico,10308.0,7.0,1996-09-18,3.0
2,3,Antonio Moreno Taquería,Antonio Moreno,Mataderos 2312,México D.F.,5023.0,Mexico,NaN,NaN,NaN,NaN
3,37,NaN,NaN,NaN,NaN,NaN,NaN,10309.0,3.0,1996-09-19,1.0
4,77,NaN,NaN,NaN,NaN,NaN,NaN,10310.0,8.0,1996-09-20,2.0


### Query 30: *`INNER JOIN WITH WHERE`* - Find the name of all reviewers who have rated their ratings with a NULL value.

> * **SQL** 

In [0]:
table_name1 = 'reviewer'
table_name2 = 'rating'

In [29]:
rs = con.execute(f'''SELECT {table_name1}.rev_name
                     FROM {table_name1}  AS {table_name1}
                     INNER JOIN {table_name2} AS {table_name2}
                     ON {table_name1}.rev_id = {table_name2}.rev_id
                     WHERE {table_name2}.rev_stars IS NULL
                     ''')
pd.DataFrame(rs.fetchall(), columns=['rev_name'])

,rev_name
0,NealWruck
1,ScottLeBrun


> * **Pandas** 

In [30]:
table1 = db[table_name1]
table2 = db[table_name2]

JOIN = pd.merge
INNER = 'inner'    
WHERE_CONDITION = (table2['rev_stars'].isnull())
WHERE_CLAUSE = WHERE_CONDITION

FROM_JOIN_WHERE = JOIN(table1, table2[WHERE_CLAUSE], INNER)

SELECT_CLAUSE = ['rev_name']

res = FROM_JOIN_WHERE[SELECT_CLAUSE]
res

,rev_name
0,NealWruck
1,ScottLeBrun


### Query 30: *`LEFT JOIN WITH WHERE`* - Find the list the first and last names of all the actors who were cast in the movie 'Annie Hall', and the roles they played in that production.

> * **SQL** - LEFT JOIN between movie and movie_cast

In [0]:
table_name1, table_name2, table_name3 = 'actor', 'movie_cast', 'movie'

In [32]:
rs = con.execute(f'''SELECT {table_name1}.act_fname, {table_name1}.act_lname, act_role.role
                     FROM {table_name1} AS {table_name1}
                     INNER JOIN (SELECT {table_name2}.act_id AS act_id, {table_name2}.role AS role
                                 FROM {table_name3}  AS {table_name3}
                                 LEFT JOIN {table_name2} AS {table_name2}
                                 ON {table_name3}.mov_id = {table_name2}.mov_id
                                 WHERE {table_name3}.mov_title = 'AnnieHall') AS act_role
                     ON act_role.act_id = {table_name1}.act_id
                     ''')
pd.DataFrame(rs.fetchall(), columns=['act_fname', 'act_lname', 'role'])

,act_fname,act_lname,role
0,Woody,Allen,AlvySinger


> * **Pandas** - LEFT JOIN between movie and movie_cast

In [33]:
table1 = db[table_name1]
table2 = db[table_name2]
table3 = db[table_name3]

JOIN = pd.merge
LEFT = 'left'    
WHERE_CONDITION = (table3['mov_title'] ==  'AnnieHall')
WHERE_CLAUSE = WHERE_CONDITION
ON = 'mov_id'
FROM_JOIN_WHERE = JOIN(table3[WHERE_CLAUSE], table2, LEFT, ON)

SELECT_CLAUSE = ['act_id', 'role']

new_table = FROM_JOIN_WHERE[SELECT_CLAUSE]

SELECT_CLAUSE1 = ['act_fname', 'act_lname', 'role']
INNER = 'inner'
ON = 'act_id'
FROM_JOIN = JOIN(table1, new_table, INNER)
res = FROM_JOIN[SELECT_CLAUSE1]
res

,act_fname,act_lname,role
0,Woody,Allen,AlvySinger


### Query 31: *`RIGHT JOIN WITH WHERE`* - Write a query to list all the actors who have acted in any movie between 1990 and 2000.

> * **SQL** - The result of RIGHT JOIN between 'movie_cast' and 'movie' is equivalent to that of LEFT JOIN between 'movie' and 'movie_cast'.

In [0]:
table_name1, table_name2, table_name3 = 'actor', 'movie_cast', 'movie'

In [35]:
rs = con.execute(f''' SELECT {table_name1}.act_fname, {table_name1}.act_lname, act_movie.mov_title,  act_movie.mov_year
                     FROM (SELECT act_id, mov_year, mov_title
                           FROM {table_name3}  AS {table_name3}
                           LEFT JOIN {table_name2} AS {table_name2}
                           ON {table_name3}.mov_id = {table_name2}.mov_id
                           WHERE {table_name3}.mov_year BETWEEN 1990 AND 2000) as act_movie
                     LEFT JOIN  {table_name1}  AS {table_name1}
                     ON {table_name1}.act_id = act_movie.act_id
                  ''')
pd.DataFrame(rs.fetchall(), columns = ['act_fname', 'act_lname', 'mov_title', 'mov_year'])

,act_fname,act_lname,mov_title,mov_year
0,Nicole,Kidman,EyesWideShut,1999
1,Stephen,Baldwin,TheUsualSuspects,1995
2,Mark,Wahlberg,BoogieNights,1997
3,Claire,Danes,PrincessMononoke,1997
4,Tim,Robbins,TheShawshankRedemption,1994
5,Kevin,Spacey,AmericanBeauty,1999
6,Kate,Winslet,Titanic,1997
7,Robin,Williams,GoodWillHunting,1997
8,Ewan,McGregor,Trainspotting,1996
9,None,None,Braveheart,1995


In [36]:
rs = con.execute(f''' SELECT {table_name1}.act_fname, {table_name1}.act_lname, {table_name3}.mov_title,  {table_name3}.mov_year
                      FROM {table_name3}  AS {table_name3} 
                      LEFT JOIN {table_name2} AS {table_name2}                       
                      ON {table_name3}.mov_id = {table_name2}.mov_id
                      LEFT JOIN  {table_name1}  AS {table_name1}                       
                      ON {table_name1}.act_id = {table_name2}.act_id
                      WHERE {table_name3}.mov_year BETWEEN 1990 AND 2000
                 ''')
pd.DataFrame(rs.fetchall(), columns = ['act_fname', 'act_lname', 'mov_title', 'mov_year'])

,act_fname,act_lname,mov_title,mov_year
0,Nicole,Kidman,EyesWideShut,1999
1,Stephen,Baldwin,TheUsualSuspects,1995
2,Mark,Wahlberg,BoogieNights,1997
3,Claire,Danes,PrincessMononoke,1997
4,Tim,Robbins,TheShawshankRedemption,1994
5,Kevin,Spacey,AmericanBeauty,1999
6,Kate,Winslet,Titanic,1997
7,Robin,Williams,GoodWillHunting,1997
8,Ewan,McGregor,Trainspotting,1996
9,None,None,Braveheart,1995


> * **Pandas** 

In [37]:
table1 = db[table_name1]
table2 = db[table_name2]
table3 = db[table_name3]

JOIN = pd.merge
RIGHT = 'right'    
WHERE_CONDITION = (table3['mov_year'].isin(range(1990, 2001, 1)))
WHERE_CLAUSE = WHERE_CONDITION
ON = 'mov_id'
FROM_JOIN_WHERE = JOIN(table2, table3[WHERE_CLAUSE], RIGHT, ON)

SELECT_CLAUSE = ['act_id', 'mov_year', 'mov_title']

new_table = FROM_JOIN_WHERE[SELECT_CLAUSE]

SELECT_CLAUSE1 = ['act_fname', 'act_lname', 'mov_title', 'mov_year']
RIGHT = 'right'
ON = 'act_id'
FROM_JOIN = JOIN(table1, new_table, RIGHT)
res = FROM_JOIN[SELECT_CLAUSE1]
res

,act_fname,act_lname,mov_title,mov_year
0,Nicole,Kidman,EyesWideShut,1999
1,Stephen,Baldwin,TheUsualSuspects,1995
2,Mark,Wahlberg,BoogieNights,1997
3,Claire,Danes,PrincessMononoke,1997
4,Tim,Robbins,TheShawshankRedemption,1994
5,Kevin,Spacey,AmericanBeauty,1999
6,Kate,Winslet,Titanic,1997
7,Robin,Williams,GoodWillHunting,1997
8,Ewan,McGregor,Trainspotting,1996
9,NaN,NaN,Braveheart,1995


In [38]:
table2.merge(table3, 'right', 'mov_id').merge(table1, 'right', 'act_id')[SELECT_CLAUSE1]

,act_fname,act_lname,mov_title,mov_year
0,James,Stewart,Vertigo,1958.0
1,Deborah,Kerr,TheInnocents,1961.0
2,Peter,OToole,LawrenceofArabia,1962.0
3,Robert,DeNiro,TheDeerHunter,1978.0
4,F.Murray,Abraham,Amadeus,1984.0
5,Harrison,Ford,BladeRunner,1982.0
6,Nicole,Kidman,EyesWideShut,1999.0
7,Stephen,Baldwin,TheUsualSuspects,1995.0
8,Mark,Wahlberg,BoogieNights,1997.0
9,Woody,Allen,AnnieHall,1977.0


### Query 32: Write a query to find the name of movie and director (first and last names) who directed a movie that casted the roles that start with 'J'.


> * **SQL**

In [0]:
table_name1 = 'director'
table_name2 = 'movie_direction'
table_name3 = 'movie_cast'
table_name4 = 'movie'

In [40]:
rs = con.execute(f'''SELECT cast_movie.mov_title, dir_movie.dir_fname, dir_movie.dir_lname
                     FROM (SELECT {table_name1}.dir_id, {table_name2}.mov_id, {table_name1}.dir_fname, {table_name1}.dir_lname
                           FROM {table_name1}  AS {table_name1}
                           LEFT JOIN {table_name2} AS {table_name2}
                           ON {table_name1}.dir_id = {table_name2}.dir_id) AS dir_movie
                     INNER JOIN
                          (SELECT {table_name4}.mov_id, {table_name4}.mov_title
                           FROM {table_name4}  AS {table_name4}
                           LEFT JOIN {table_name3} AS {table_name3}
                           ON {table_name3}.mov_id = {table_name4}.mov_id
                           WHERE {table_name3}.role LIKE "J%") AS cast_movie 
                     ON dir_movie.mov_id = cast_movie.mov_id                     
                     ''')
pd.DataFrame(rs.fetchall(), columns = ['mov_title', 'dir_fname', 'dir_lname'])

,mov_title,dir_fname,dir_lname
0,Vertigo,Alfred,Hitchcock
1,Chinatown,Roman,Polanski


In [41]:
rs = con.execute(f'''SELECT {table_name4}.mov_title, {table_name1}.dir_fname, {table_name1}.dir_lname
                     FROM {table_name1}  AS {table_name1}
                     LEFT JOIN {table_name2} AS {table_name2}                                
                     ON {table_name1}.dir_id = {table_name2}.dir_id
                     INNER JOIN {table_name4}  AS {table_name4}                                                   
                     ON {table_name2}.mov_id = {table_name4}.mov_id                     
                     LEFT JOIN {table_name3} AS {table_name3}             
                     ON {table_name3}.mov_id = {table_name4}.mov_id
                     WHERE {table_name3}.role LIKE "J%"                      
                     ''')
pd.DataFrame(rs.fetchall(), columns = ['mov_title', 'dir_fname', 'dir_lname'])

,mov_title,dir_fname,dir_lname
0,Vertigo,Alfred,Hitchcock
1,Chinatown,Roman,Polanski


> * **Pandas** 

In [42]:
table1, table2 = db[table_name1], db[table_name2]
table3, table4 = db[table_name3], db[table_name4]

# sub query 1
JOIN = pd.merge
LEFT_OUTER = 'left'    
ON = 'dir_id'
FROM_CLAUSE_WITH_JOIN1 = JOIN(table1, table2, LEFT_OUTER, ON)
SELECT_CLAUSE1 = ['dir_id', 'mov_id', 'dir_fname', 'dir_lname']
res1 = FROM_CLAUSE_WITH_JOIN1[SELECT_CLAUSE1]

# sub query 2
RIGHT_OUTER = 'left'    
ON = 'mov_id'
WHERE_CLAUSE = (table3['role'].str.startswith("J"))
FROM_CLAUSE_WITH_JOIN2 = JOIN(table3[WHERE_CLAUSE], table4, RIGHT_OUTER, ON)
SELECT_CLAUSE2 = ['mov_id', 'mov_title']
res2 = FROM_CLAUSE_WITH_JOIN2[SELECT_CLAUSE2]

# final query
INNER = 'inner'    
ON = 'mov_id'
FROM_CLAUSE_WITH_JOIN = JOIN(res1, res2, INNER, ON)
SELECT_CLAUSE = ['mov_title', 'dir_fname', 'dir_lname']
rs = FROM_CLAUSE_WITH_JOIN[SELECT_CLAUSE]
rs

,mov_title,dir_fname,dir_lname
0,Vertigo,Alfred,Hitchcock
1,Chinatown,Roman,Polanski


In [43]:
table1.merge(right=table2, how='left', on='dir_id') \
      .merge(table3[table3['role'].str.startswith("J")], 'inner', 'mov_id') \
      .merge(table4, 'left', 'mov_id')[SELECT_CLAUSE]

,mov_title,dir_fname,dir_lname
0,Vertigo,Alfred,Hitchcock
1,Chinatown,Roman,Polanski


### Query 33: Write a query to find the name of movie and director (first and last names) who directed a movie that casted a role as 'Sean Maguire'.


In [0]:
table_name1 = 'director'
table_name2 = 'movie_direction'
table_name3 = 'movie_cast'
table_name4 = 'movie'

> * **SQL**

In [45]:
rs = con.execute(f'''SELECT cast_movie.mov_title, dir_movie.dir_fname, dir_movie.dir_lname
                     FROM (SELECT {table_name1}.dir_id, {table_name2}.mov_id, {table_name1}.dir_fname, {table_name1}.dir_lname
                           FROM {table_name1}  AS {table_name1}
                           LEFT JOIN {table_name2} AS {table_name2}
                           ON {table_name1}.dir_id = {table_name2}.dir_id) AS dir_movie
                     INNER JOIN
                          (SELECT {table_name4}.mov_id, {table_name4}.mov_title
                           FROM {table_name4}  AS {table_name4}
                           LEFT JOIN {table_name3} AS {table_name3}
                           ON {table_name3}.mov_id = {table_name4}.mov_id
                           WHERE {table_name3}.role = "SeanMaguire") AS cast_movie 
                     ON dir_movie.mov_id = cast_movie.mov_id                     
                     ''')
pd.DataFrame(rs.fetchall(), columns = ['mov_title', 'dir_fname', 'dir_lname'])

,mov_title,dir_fname,dir_lname
0,GoodWillHunting,Gus,VanSant


In [46]:
rs = con.execute(f'''SELECT {table_name4}.mov_title,  {table_name1}.dir_fname,  {table_name1}.dir_lname
                     FROM {table_name1} AS {table_name1}                           
                     LEFT JOIN {table_name2} AS {table_name2}
                     ON {table_name1}.dir_id = {table_name2}.dir_id
                     INNER JOIN {table_name4} AS {table_name4}         
                     ON  {table_name2}.mov_id = {table_name4}.mov_id                     
                     LEFT JOIN {table_name3} AS {table_name3}
                     ON {table_name3}.mov_id = {table_name4}.mov_id
                     WHERE {table_name3}.role = "SeanMaguire"                    
                  ''')
pd.DataFrame(rs.fetchall(), columns = ['mov_title', 'dir_fname', 'dir_lname'])

,mov_title,dir_fname,dir_lname
0,GoodWillHunting,Gus,VanSant


> * **Pandas** 

In [47]:
table1, table2 = db[table_name1], db[table_name2]
table3, table4 = db[table_name3], db[table_name4]

# sub query 1
JOIN = pd.merge
LEFT_OUTER = 'left'    
ON = 'dir_id'
FROM_CLAUSE_WITH_JOIN1 = JOIN(table1, table2, LEFT_OUTER, ON)
SELECT_CLAUSE1 = ['dir_id', 'mov_id', 'dir_fname', 'dir_lname']
res1 = FROM_CLAUSE_WITH_JOIN1[SELECT_CLAUSE1]

# sub query 2
RIGHT_OUTER = 'left'    
ON = 'mov_id'
WHERE_CLAUSE = (table3['role'] == "SeanMaguire")
FROM_CLAUSE_WITH_JOIN2 = JOIN(table3[WHERE_CLAUSE], table4, RIGHT_OUTER, ON)
SELECT_CLAUSE2 = ['mov_id', 'mov_title']
res2 = FROM_CLAUSE_WITH_JOIN2[SELECT_CLAUSE2]

# final query
INNER = 'inner'    
ON = 'mov_id'
FROM_CLAUSE_WITH_JOIN = JOIN(res1, res2, INNER, ON)
SELECT_CLAUSE = ['mov_title', 'dir_fname', 'dir_lname']
rs = FROM_CLAUSE_WITH_JOIN[SELECT_CLAUSE]
rs

,mov_title,dir_fname,dir_lname
0,GoodWillHunting,Gus,VanSant


In [48]:
table1.merge(right=table2, how='left', on='dir_id') \
      .merge(table3[table3['role'] == "SeanMaguire"], 'inner', 'mov_id') \
      .merge(table4, 'left', 'mov_id')[SELECT_CLAUSE]

,mov_title,dir_fname,dir_lname
0,GoodWillHunting,Gus,VanSant


### Query 34: Write a query to list first and last name of all the directors with number of genres movies the directed with genres name, and arranged the result alphabetically with the first and last name of the director.

In [0]:
table_name1, table_name2  = 'director', 'movie_direction'
table_name3, table_name4 = 'movie_genres', 'genres'

> * **SQL**

In [50]:
rs = con.execute(f'''SELECT {table_name1}.dir_fname,  {table_name1}.dir_lname, {table_name4}.gen_title, COUNT({table_name4}.gen_title)
                     FROM {table_name1} AS {table_name1}                           
                     INNER JOIN {table_name2} AS {table_name2}
                     ON {table_name1}.dir_id = {table_name2}.dir_id
                     INNER JOIN {table_name3} AS {table_name3}         
                     ON  {table_name2}.mov_id = {table_name3}.mov_id                     
                     INNER JOIN {table_name4} AS {table_name4}
                     ON {table_name3}.gen_id = {table_name4}.gen_id
                     GROUP BY {table_name1}.dir_fname, {table_name1}.dir_lname, {table_name4}.gen_title
                     ORDER BY {table_name1}.dir_fname, {table_name1}.dir_lname
                  ''')
pd.DataFrame(rs.fetchall(), columns = ['dir_fname', 'dir_lname', 'gen_title', 'count_gen_title']).T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
dir_fname,Alfred,Bryan,Danny,David,Frank,Hayao,Jack,James,John,Kevin,Michael,Ridley,Sam,Stanley,Woody
dir_lname,Hitchcock,Singer,Boyle,Lean,Darabont,Miyazaki,Clayton,Cameron,Boorman,Spacey,Cimino,Scott,Mendes,Kubrick,Allen
gen_title,Mystery,Crime,Drama,Adventure,Crime,Animation,Horror,Action,Adventure,Music,War,Thriller,Romance,Mystery,Comedy
count_gen_title,1,1,2,1,1,1,1,1,1,1,1,1,1,1,1


> * **Pandas**

In [51]:
table1, table2 = db[table_name1], db[table_name2]
table3, table4 = db[table_name3], db[table_name4]

ORDER_BY_COLUMNS = ['dir_fname', 'dir_lname']
GROUP_BY_COLUMNS = ['dir_fname', 'dir_lname', 'gen_title']
AGG_OP = {'gen_title': 'count'}
SELECT_CLAUSE = ['dir_fname', 'dir_lname', 'gen_title', 'count_gen_title']
rs = table1.merge(right=table2, how='inner', on='dir_id') \
      .merge(table3, 'inner', 'mov_id') \
      .merge(table4, 'inner', 'gen_id') \
      .groupby(GROUP_BY_COLUMNS, sort=ORDER_BY_COLUMNS).agg(AGG_OP) 
rs.columns = ['count_gen_title']
rs.reset_index(drop=False).T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
dir_fname,Alfred,Bryan,Danny,David,Frank,Hayao,Jack,James,John,Kevin,Michael,Ridley,Sam,Stanley,Woody
dir_lname,Hitchcock,Singer,Boyle,Lean,Darabont,Miyazaki,Clayton,Cameron,Boorman,Spacey,Cimino,Scott,Mendes,Kubrick,Allen
gen_title,Mystery,Crime,Drama,Adventure,Crime,Animation,Horror,Action,Adventure,Music,War,Thriller,Romance,Mystery,Comedy
count_gen_title,1,1,2,1,1,1,1,1,1,1,1,1,1,1,1


### Query 35: Write a query in to list all the movies with year and genres.

In [0]:
table_name1, table_name2, table_name3  = 'movie',  'movie_genres',  'genres'

> * **SQL**

In [53]:
rs = con.execute(f''' SELECT {table_name1}.mov_title, {table_name1}.mov_year, {table_name3}.gen_title
                      FROM {table_name1}  AS {table_name1} 
                      LEFT JOIN {table_name2} AS {table_name2}                       
                      ON {table_name1}.mov_id = {table_name2}.mov_id
                      LEFT JOIN  {table_name3}  AS {table_name3}                       
                      ON {table_name2}.gen_id = {table_name3}.gen_id                      
                 ''')
pd.DataFrame(rs.fetchall(), columns = ['mov_title', 'mov_year', 'gen_title']).T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27
mov_title,Vertigo,TheInnocents,LawrenceofArabia,TheDeerHunter,Amadeus,BladeRunner,EyesWideShut,TheUsualSuspects,Chinatown,BoogieNights,AnnieHall,PrincessMononoke,TheShawshankRedemption,AmericanBeauty,Titanic,GoodWillHunting,Deliverance,Trainspotting,ThePrestige,DonnieDarko,SlumdogMillionaire,Aliens,BeyondtheSea,Avatar,SevenSamurai,SpiritedAway,BacktotheFuture,Braveheart
mov_year,1958,1961,1962,1978,1984,1982,1999,1995,1974,1997,1977,1997,1994,1999,1997,1997,1972,1996,2006,2001,2008,1986,2004,2009,1954,2001,1985,1995
gen_title,Mystery,Horror,Adventure,War,None,Thriller,Mystery,Crime,None,None,Comedy,Animation,Crime,Romance,None,None,Adventure,Drama,None,None,Drama,Action,Music,None,Drama,Mystery,Drama,None


> * **Pandas**

In [54]:
table1, table2 = db[table_name1], db[table_name2]
table3 = db[table_name3]

SELECT_CLAUSE = ['mov_title', 'mov_year', 'gen_title']
rs = table1.merge(right=table2, how='left', on='mov_id') \
      .merge(table3, 'left', 'gen_id')[SELECT_CLAUSE]
rs.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27
mov_title,Vertigo,TheInnocents,LawrenceofArabia,TheDeerHunter,Amadeus,BladeRunner,EyesWideShut,TheUsualSuspects,Chinatown,BoogieNights,AnnieHall,PrincessMononoke,TheShawshankRedemption,AmericanBeauty,Titanic,GoodWillHunting,Deliverance,Trainspotting,ThePrestige,DonnieDarko,SlumdogMillionaire,Aliens,BeyondtheSea,Avatar,SevenSamurai,SpiritedAway,BacktotheFuture,Braveheart
mov_year,1958,1961,1962,1978,1984,1982,1999,1995,1974,1997,1977,1997,1994,1999,1997,1997,1972,1996,2006,2001,2008,1986,2004,2009,1954,2001,1985,1995
gen_title,Mystery,Horror,Adventure,War,NaN,Thriller,Mystery,Crime,NaN,NaN,Comedy,Animation,Crime,Romance,NaN,NaN,Adventure,Drama,NaN,NaN,Drama,Action,Music,NaN,Drama,Mystery,Drama,NaN


### Query 36: Write a query to list all the movies with year, genres, and name of the director.

In [0]:
table_name1, table_name2, table_name3, table_name4, table_name5  = 'movie',  'movie_genres',  'genres', 'movie_direction', 'director'

> * **SQL**

In [56]:
rs = con.execute(f''' SELECT {table_name1}.mov_title, {table_name1}.mov_year, {table_name3}.gen_title,
                             {table_name5}.dir_fname, {table_name5}.dir_lname
                      FROM {table_name1}  AS {table_name1} 
                      LEFT JOIN {table_name2} AS {table_name2}                       
                      ON {table_name1}.mov_id = {table_name2}.mov_id
                      LEFT JOIN  {table_name3}  AS {table_name3}                       
                      ON {table_name2}.gen_id = {table_name3}.gen_id                      
                      LEFT JOIN  {table_name4}  AS {table_name4}                       
                      ON {table_name1}.mov_id = {table_name4}.mov_id                      
                      LEFT JOIN  {table_name5}  AS {table_name5}                       
                      ON {table_name4}.dir_id = {table_name5}.dir_id                      
                 ''')
pd.DataFrame(rs.fetchall(), columns = ['mov_title', 'mov_year', 'gen_title', 'dir_fname', 'dir_lname']).T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27
mov_title,Vertigo,TheInnocents,LawrenceofArabia,TheDeerHunter,Amadeus,BladeRunner,EyesWideShut,TheUsualSuspects,Chinatown,BoogieNights,AnnieHall,PrincessMononoke,TheShawshankRedemption,AmericanBeauty,Titanic,GoodWillHunting,Deliverance,Trainspotting,ThePrestige,DonnieDarko,SlumdogMillionaire,Aliens,BeyondtheSea,Avatar,SevenSamurai,SpiritedAway,BacktotheFuture,Braveheart
mov_year,1958,1961,1962,1978,1984,1982,1999,1995,1974,1997,1977,1997,1994,1999,1997,1997,1972,1996,2006,2001,2008,1986,2004,2009,1954,2001,1985,1995
gen_title,Mystery,Horror,Adventure,War,None,Thriller,Mystery,Crime,None,None,Comedy,Animation,Crime,Romance,None,None,Adventure,Drama,None,None,Drama,Action,Music,None,Drama,Mystery,Drama,None
dir_fname,Alfred,Jack,David,Michael,Milos,Ridley,Stanley,Bryan,Roman,Paul,Woody,Hayao,Frank,Sam,James,Gus,John,Danny,Christopher,Richard,Danny,James,Kevin,None,None,None,None,None
dir_lname,Hitchcock,Clayton,Lean,Cimino,Forman,Scott,Kubrick,Singer,Polanski,ThomasAnderson,Allen,Miyazaki,Darabont,Mendes,Cameron,VanSant,Boorman,Boyle,Nolan,Kelly,Boyle,Cameron,Spacey,None,None,None,None,None


> * **Pandas**

In [57]:
table1, table2, table3 = db[table_name1], db[table_name2], db[table_name3]
table4, table5 = db[table_name4], db[table_name5]

SELECT_CLAUSE = ['mov_title', 'mov_year', 'gen_title', 'dir_fname', 'dir_lname']
rs = table1.merge(right=table2, how='left', on='mov_id') \
      .merge(table3, 'left', 'gen_id') \
      .merge(table4, 'left', 'mov_id') \
      .merge(table5, 'left', 'dir_id') \
      [SELECT_CLAUSE]
rs.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27
mov_title,Vertigo,TheInnocents,LawrenceofArabia,TheDeerHunter,Amadeus,BladeRunner,EyesWideShut,TheUsualSuspects,Chinatown,BoogieNights,AnnieHall,PrincessMononoke,TheShawshankRedemption,AmericanBeauty,Titanic,GoodWillHunting,Deliverance,Trainspotting,ThePrestige,DonnieDarko,SlumdogMillionaire,Aliens,BeyondtheSea,Avatar,SevenSamurai,SpiritedAway,BacktotheFuture,Braveheart
mov_year,1958,1961,1962,1978,1984,1982,1999,1995,1974,1997,1977,1997,1994,1999,1997,1997,1972,1996,2006,2001,2008,1986,2004,2009,1954,2001,1985,1995
gen_title,Mystery,Horror,Adventure,War,NaN,Thriller,Mystery,Crime,NaN,NaN,Comedy,Animation,Crime,Romance,NaN,NaN,Adventure,Drama,NaN,NaN,Drama,Action,Music,NaN,Drama,Mystery,Drama,NaN
dir_fname,Alfred,Jack,David,Michael,Milos,Ridley,Stanley,Bryan,Roman,Paul,Woody,Hayao,Frank,Sam,James,Gus,John,Danny,Christopher,Richard,Danny,James,Kevin,NaN,NaN,NaN,NaN,NaN
dir_lname,Hitchcock,Clayton,Lean,Cimino,Forman,Scott,Kubrick,Singer,Polanski,ThomasAnderson,Allen,Miyazaki,Darabont,Mendes,Cameron,VanSant,Boorman,Boyle,Nolan,Kelly,Boyle,Cameron,Spacey,NaN,NaN,NaN,NaN,NaN


### Query 37: Write a query to list all the movies with title, year, date of release, movie duration, and first and last name of the director which released before 1st january 1989, and sort the result set according to release date from highest date to lowest. 

In [0]:
table_name1, table_name2, table_name3  = 'movie', 'movie_direction', 'director'

> * **SQL**

In [59]:
rs = con.execute(f''' SELECT {table_name1}.mov_title, {table_name1}.mov_year, {table_name1}.mov_dt_rel,
                             {table_name5}.dir_fname, {table_name5}.dir_lname
                      FROM {table_name1}  AS {table_name1} 
                      LEFT JOIN {table_name2} AS {table_name2}                       
                      ON {table_name1}.mov_id = {table_name2}.mov_id
                      LEFT JOIN  {table_name3}  AS {table_name3}                       
                      ON {table_name2}.dir_id = {table_name3}.dir_id
                      WHERE {table_name1}.mov_dt_rel < "1989-01-01"
                      ORDER BY {table_name1}.mov_dt_rel DESC
                 ''')
pd.DataFrame(rs.fetchall(), columns = ['mov_title', 'mov_year', 'mov_dt_rel', 'dir_fname', 'dir_lname']).T

,0,1,2,3,4,5,6,7,8,9,10,11
mov_title,Aliens,BacktotheFuture,Amadeus,Deliverance,BladeRunner,TheDeerHunter,AnnieHall,Chinatown,LawrenceofArabia,TheInnocents,Vertigo,SevenSamurai
mov_year,1986,1985,1984,1972,1982,1978,1977,1974,1962,1961,1958,1954
mov_dt_rel,1986-08-29,1985-12-04,1985-01-07,1982-10-05,1982-09-09,1979-03-08,1977-04-20,1974-08-09,1962-12-11,1962-02-19,1958-08-24,1954-04-26
dir_fname,James,None,Milos,John,Ridley,Michael,Woody,Roman,David,Jack,Alfred,None
dir_lname,Cameron,None,Forman,Boorman,Scott,Cimino,Allen,Polanski,Lean,Clayton,Hitchcock,None


> * **Pandas**

In [60]:
table1, table2, table3 = db[table_name1], db[table_name2], db[table_name3]

SELECT_CLAUSE = ['mov_title', 'mov_year', 'mov_dt_rel', 'dir_fname', 'dir_lname']
WHERE_CLAUSE = (table1['mov_dt_rel'] < '1989-01-01')
rs = table1[WHERE_CLAUSE].merge(right=table2, how='left', on='mov_id') \
      .merge(table3, 'left', 'dir_id') \
      [SELECT_CLAUSE].sort_values(['mov_dt_rel'], ascending=False)
rs.T

,9,11,4,8,5,3,7,6,2,1,0,10
mov_title,Aliens,BacktotheFuture,Amadeus,Deliverance,BladeRunner,TheDeerHunter,AnnieHall,Chinatown,LawrenceofArabia,TheInnocents,Vertigo,SevenSamurai
mov_year,1986,1985,1984,1972,1982,1978,1977,1974,1962,1961,1958,1954
mov_dt_rel,1986-08-29,1985-12-04,1985-01-07,1982-10-05,1982-09-09,1979-03-08,1977-04-20,1974-08-09,1962-12-11,1962-02-19,1958-08-24,1954-04-26
dir_fname,James,NaN,Milos,John,Ridley,Michael,Woody,Roman,David,Jack,Alfred,NaN
dir_lname,Cameron,NaN,Forman,Boorman,Scott,Cimino,Allen,Polanski,Lean,Clayton,Hitchcock,NaN


### Query 38: Write a query in SQL to compute a report which contain the genres of those movies with their average time and number of movies for each genres.

In [0]:
table_name1, table_name2, table_name3  = 'movie', 'movie_genres', 'genres'

> * **SQL**

In [62]:
rs = con.execute(f''' SELECT {table_name3}.gen_title, COUNT({table_name1}.mov_id), AVG({table_name1}.mov_time)                            
                      FROM {table_name1}  AS {table_name1} 
                      LEFT JOIN {table_name2} AS {table_name2}                       
                      ON {table_name1}.mov_id = {table_name2}.mov_id
                      LEFT JOIN  {table_name3}  AS {table_name3}                       
                      ON {table_name2}.gen_id = {table_name3}.gen_id
                      GROUP BY {table_name3}.gen_title                      
                 ''')
pd.DataFrame(rs.fetchall(), columns = ['gen_title', 'movie_count', 'avg_movie_time']).T

,0,1,2,3,4,5,6,7,8,9,10,11,12
gen_title,None,Action,Adventure,Animation,Comedy,Crime,Drama,Horror,Music,Mystery,Romance,Thriller,War
movie_count,9,1,2,1,1,2,4,1,1,3,1,1,1
avg_movie_time,149.778,137,162.5,134,93,124,134.25,100,118,137.333,122,117,183


> * There are 9 movies for which we don't know the genres.

> * **Pandas**

In [63]:
table1, table2, table3 = db[table_name1], db[table_name2], db[table_name3]

SELECT_CLAUSE = ['gen_title', 'movie_count', 'avg_movie_time']
GROUPBY_COLUMNS = ['gen_title']
AGG_OP = {'mov_id':'count', 'mov_time': 'mean'}
rs = table1.merge(right=table2, how='left', on='mov_id') \
      .merge(table3, 'left', 'gen_id') \
      .groupby(GROUPBY_COLUMNS) \
      .agg(AGG_OP)
rs

,mov_id,mov_time
gen_title,,
Action,1,137.000000
Adventure,2,162.500000
Animation,1,134.000000
Comedy,1,93.000000
Crime,2,124.000000
Drama,4,134.250000
Horror,1,100.000000
Music,1,118.000000
Mystery,3,137.333333


### Query 39:  Write a query to find the movie with lowest duration along with the year, director's name, actor's name and his/her role in that production.

In [0]:
table_name1, table_name2, table_name3, table_name4, table_name5  = 'movie',  'movie_direction', 'director', 'movie_cast', 'actor'

> * **SQL**

In [65]:
rs = con.execute(f''' SELECT {table_name1}.mov_title, {table_name1}.mov_time, {table_name1}.mov_year,
                             {table_name3}.dir_fname, {table_name3}.dir_lname, {table_name5}.act_fname,
                             {table_name5}.act_lname, {table_name4}.role
                      FROM {table_name1} AS {table_name1} 
                      LEFT JOIN {table_name2} AS {table_name2}                       
                      ON {table_name1}.mov_id = {table_name2}.mov_id
                      LEFT JOIN  {table_name3} AS {table_name3}                       
                      ON {table_name2}.dir_id = {table_name3}.dir_id
                      LEFT JOIN  {table_name4} AS {table_name4}                       
                      ON {table_name2}.mov_id = {table_name4}.mov_id
                      LEFT JOIN  {table_name5} AS {table_name5}
                      ON {table_name4}.act_id = {table_name5}.act_id
                      ORDER BY {table_name1}.mov_time ASC LIMIT 1
                 ''')
pd.DataFrame(rs.fetchall(), columns = ['mov_title', 'lowest_duration', 'mov_year',
                                       'dir_fname', 'dir_lname', 'act_fname',
                                       'act_lname', 'role'])

,mov_title,lowest_duration,mov_year,dir_fname,dir_lname,act_fname,act_lname,role
0,AnnieHall,93,1977,Woody,Allen,Woody,Allen,AlvySinger


> * **Pandas**

In [66]:
table1, table2, table3 = db[table_name1], db[table_name2], db[table_name3]
table4, table5 = db[table_name4], db[table_name5]

SELECT_CLAUSE = ['mov_title', 'mov_year',
                 'dir_fname', 'dir_lname', 'act_fname',     
                 'act_lname', 'role']

rs = table1.merge(right=table2, how='left', on='mov_id') \
      .merge(table3, 'left', 'dir_id') \
      .merge(table4, 'left', 'mov_id') \
      .merge(table5, 'left', 'act_id') \
      .sort_values(['mov_time'], ascending=True)[SELECT_CLAUSE].head(1)
rs

,mov_title,mov_year,dir_fname,dir_lname,act_fname,act_lname,role
10,AnnieHall,1977,Woody,Allen,Woody,Allen,AlvySinger


### Query 40: Write a query to find all the years which produced a movie that received a rating of 3 or 4, and sort the result in increasing order.

In [0]:
table_name1, table_name2 = 'movie',  'rating'

> * **SQL**

In [68]:
rs = con.execute(f''' SELECT {table_name1}.mov_year, {table_name2}.rev_stars
                      FROM {table_name1} AS {table_name1} 
                      INNER JOIN {table_name2} AS {table_name2}                       
                      ON {table_name1}.mov_id = {table_name2}.mov_id
                      WHERE {table_name2}.rev_stars = 3.00 OR {table_name2}.rev_stars = 4.00
                      ORDER BY {table_name2}.rev_stars ASC
                 ''')
pd.DataFrame(rs.fetchall(), columns = ['mov_year', 'rev_stars'])

,mov_year,rev_stars
0,1997,3.0
1,1997,4.0


> * **Pandas**

In [69]:
table1, table2 = db[table_name1], db[table_name2]

SELECT_CLAUSE = ['mov_year', 'rev_stars']
WHERE_CLAUSE = ((table2['rev_stars'] == 3.00) | (table2['rev_stars'] == 4.00))
rs = table1.merge(right=table2[WHERE_CLAUSE], how='inner', on='mov_id') \
           .sort_values(['rev_stars'], ascending=True)[SELECT_CLAUSE]
rs

,mov_year,rev_stars
0,1997,3.0
1,1997,4.0


### Query 41: Write a query in to return the reviewer name, movie title, and stars in an order that reviewer name will come first, then by movie title, and lastly by number of stars.

In [0]:
table_name1, table_name2, table_name3 = 'movie',  'rating', 'reviewer'

> * **SQL**

In [71]:
rs = con.execute(f''' SELECT {table_name3}.rev_name, {table_name1}.mov_title, {table_name2}.rev_stars
                      FROM {table_name1} AS {table_name1} 
                      INNER JOIN {table_name2} AS {table_name2}                       
                      ON {table_name1}.mov_id = {table_name2}.mov_id                      
                      INNER JOIN {table_name3} AS {table_name3}                       
                      ON {table_name2}.rev_id = {table_name3}.rev_id
                 ''')
pd.DataFrame(rs.fetchall(), columns = ['rev_name', 'mov_title', 'rev_stars']).T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
rev_name,RightySock,JackMalvern,FlagrantBaronessa,None,SimonWright,NealWruck,PaulMonks,MikeSalvati,None,SashaGoldshtein,RightySock,JoshCates,ScottLeBrun,HannahSteele,VincentCadena,BrandtSponseller,RichardAdams,VictorWoeltjen,KrugStillo
mov_title,Vertigo,TheInnocents,LawrenceofArabia,BladeRunner,TheUsualSuspects,Chinatown,BoogieNights,AnnieHall,PrincessMononoke,AmericanBeauty,Titanic,GoodWillHunting,Trainspotting,DonnieDarko,SlumdogMillionaire,Aliens,BeyondtheSea,Avatar,Braveheart
rev_stars,8.4,7.9,8.3,8.2,8.6,NaN,3,8.1,8.4,7,7.7,4,NaN,8.1,8,8.4,6.7,7.3,7.7


> * **Pandas**

In [72]:
table1, table2, table3 = db[table_name1], db[table_name2], db[table_name3]

SELECT_CLAUSE =  ['rev_name', 'mov_title', 'rev_stars']

rs = table1.merge(right=table2, how='inner', on='mov_id') \
      .merge(table3, 'inner', 'rev_id')[SELECT_CLAUSE]
rs.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
rev_name,RightySock,RightySock,JackMalvern,FlagrantBaronessa,NaN,SimonWright,NealWruck,PaulMonks,MikeSalvati,NaN,SashaGoldshtein,JoshCates,ScottLeBrun,HannahSteele,VincentCadena,BrandtSponseller,RichardAdams,VictorWoeltjen,KrugStillo
mov_title,Vertigo,Titanic,TheInnocents,LawrenceofArabia,BladeRunner,TheUsualSuspects,Chinatown,BoogieNights,AnnieHall,PrincessMononoke,AmericanBeauty,GoodWillHunting,Trainspotting,DonnieDarko,SlumdogMillionaire,Aliens,BeyondtheSea,Avatar,Braveheart
rev_stars,8.4,7.7,7.9,8.3,8.2,8.6,NaN,3,8.1,8.4,7,4,NaN,8.1,8,8.4,6.7,7.3,7.7


### Query 42: Write a query to find movie title and number of stars for each movie that has at least one rating and find the highest number of stars that movie received and sort the result by movie title. 

In [0]:
table_name1, table_name2  = 'movie',  'rating'

> * **SQL**

In [74]:
rs = con.execute(f''' SELECT {table_name1}.mov_title, {table_name2}.rev_stars 
                      FROM {table_name1} AS {table_name1} 
                      INNER JOIN {table_name2} AS {table_name2}                       
                      ON {table_name1}.mov_id = {table_name2}.mov_id             
                      WHERE {table_name2}.num_of_ratings > 0 AND 
                            {table_name2}.rev_stars = (SELECT MAX(rev_stars) 
                                                       FROM  {table_name2})
                      ORDER BY {table_name1}.mov_title
                 ''')
pd.DataFrame(rs.fetchall(), columns = ['mov_title', 'rev_stars'])

,mov_title,rev_stars
0,TheUsualSuspects,8.6


> * **Pandas**

In [75]:
table1, table2 = db[table_name1], db[table_name2]

WHERE_CONDITION1 = (table2['num_of_ratings'] > 0)
WHERE_CONDITION2 = (table2['rev_stars'] == table2['rev_stars'].max())                      
WHERE_CLAUSE =  WHERE_CONDITION1 & WHERE_CONDITION2
SELECT_CLAUSE =  ['mov_title', 'rev_stars']

rs = table1.merge(right=table2[WHERE_CLAUSE], how='inner',
                  on='mov_id')[SELECT_CLAUSE]
rs

,mov_title,rev_stars
0,TheUsualSuspects,8.6


### Query 43: Write a query to find the director's first and last name together with the title of the movie(s) they directed and received the rating.

In [0]:
table_name1, table_name2, table_name3, table_name4 = 'movie',  'rating', 'movie_direction', 'director'

> * **SQL**

In [77]:
rs = con.execute(f''' SELECT {table_name4}.dir_fname, {table_name4}.dir_lname,
                             {table_name1}.mov_title, {table_name2}.num_of_ratings
                      FROM {table_name1} AS {table_name1} 
                      INNER JOIN {table_name3} AS {table_name3}                       
                      ON {table_name1}.mov_id = {table_name2}.mov_id      
                      INNER JOIN {table_name4} AS {table_name4}                       
                      ON {table_name3}.dir_id = {table_name4}.dir_id
                      INNER JOIN {table_name2} AS {table_name2}                       
                      ON {table_name3}.mov_id = {table_name2}.mov_id                                                
                 ''')
pd.DataFrame(rs.fetchall(), columns = ['dir_fname', 'dir_lname', 'mov_title', 'num_of_ratings']).T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
dir_fname,Alfred,Jack,David,Ridley,Bryan,Roman,Paul,Woody,Hayao,Sam,James,Gus,Danny,Richard,Danny,James,Kevin
dir_lname,Hitchcock,Clayton,Lean,Scott,Singer,Polanski,ThomasAnderson,Allen,Miyazaki,Mendes,Cameron,VanSant,Boyle,Kelly,Boyle,Cameron,Spacey
mov_title,Vertigo,TheInnocents,LawrenceofArabia,BladeRunner,TheUsualSuspects,Chinatown,BoogieNights,AnnieHall,PrincessMononoke,AmericanBeauty,Titanic,GoodWillHunting,Trainspotting,DonnieDarko,SlumdogMillionaire,Aliens,BeyondtheSea
num_of_ratings,263575,20207,202778,484746,779489,227235,195961,203875,NaN,862618,830095,642132,580301,609451,667758,511613,13091


> * **Pandas**

In [78]:
table1, table2 = db[table_name1], db[table_name2]
table3, table4 = db[table_name3], db[table_name4]

SELECT_CLAUSE = ['dir_fname', 'dir_lname', 'mov_title', 'num_of_ratings']

rs = table1.merge(right=table3, how='inner', on='mov_id') \
           .merge(right=table4, how='inner', on='dir_id') \
           .merge(right=table2, how='inner', on='mov_id') \
           [SELECT_CLAUSE]
rs.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
dir_fname,Alfred,Jack,David,Ridley,Bryan,Roman,Paul,Woody,Hayao,Sam,James,James,Gus,Danny,Danny,Richard,Kevin
dir_lname,Hitchcock,Clayton,Lean,Scott,Singer,Polanski,ThomasAnderson,Allen,Miyazaki,Mendes,Cameron,Cameron,VanSant,Boyle,Boyle,Kelly,Spacey
mov_title,Vertigo,TheInnocents,LawrenceofArabia,BladeRunner,TheUsualSuspects,Chinatown,BoogieNights,AnnieHall,PrincessMononoke,AmericanBeauty,Titanic,Aliens,GoodWillHunting,Trainspotting,SlumdogMillionaire,DonnieDarko,BeyondtheSea
num_of_ratings,263575,20207,202778,484746,779489,227235,195961,203875,NaN,862618,830095,511613,642132,580301,667758,609451,13091


### Query 44:  Write a query to find the movie title, actor first and last name, and the role for those movies where one or more actors acted in two or more movies.

In [0]:
table_name1, table_name2, table_name3 = 'movie',  'movie_cast', 'actor'

> * **SQL**

In [80]:
rs = con.execute(f''' SELECT {table_name1}.mov_title, {table_name3}.act_fname,
                             {table_name3}.act_lname, {table_name2}.role
                      FROM {table_name1} AS {table_name1} 
                      INNER JOIN {table_name2} AS {table_name2}                       
                      ON {table_name1}.mov_id = {table_name2}.mov_id      
                      INNER JOIN {table_name3} AS {table_name3}                       
                      ON {table_name2}.act_id = {table_name3}.act_id
                      GROUP BY {table_name2}.act_id
                      HAVING COUNT({table_name2}.mov_id) >= 2
                                                                 
                 ''')
pd.DataFrame(rs.fetchall(), columns = ['mov_title', 'act_fname', 'act_lname', 'role'])

,mov_title,act_fname,act_lname,role
0,BeyondtheSea,Kevin,Spacey,BobbyDarin


> * **Pandas**

In [81]:
table1, table2 = db[table_name1], db[table_name2]
table3 = db[table_name3]

SELECT_CLAUSE = ['mov_title', 'act_fname', 'act_lname', 'role']

rs = table1.merge(table2, 'inner', 'mov_id') \
           .merge(table3, 'inner','act_id') \
           .groupby(['act_id']).filter(lambda x: x['mov_id'].count() >= 2) \
           [SELECT_CLAUSE]
rs

,mov_title,act_fname,act_lname,role
13,AmericanBeauty,Kevin,Spacey,LesterBurnham
14,BeyondtheSea,Kevin,Spacey,BobbyDarin


### Query 45: Write a query to find the first and last name of a director and the movie he or she directed, and the actress appeared which first name was Claire and last name was Danes along with her role in that movie. 

In [0]:
table_name1, table_name2, table_name3 = 'movie',  'movie_cast', 'actor',
table_name4, table_name5 = 'movie_direction', 'director'

> * **SQL**

In [83]:
rs = con.execute(f''' SELECT {table_name5}.dir_fname, {table_name5}.dir_lname,
                             {table_name1}.mov_title, {table_name2}.role
                      FROM {table_name1} AS {table_name1} 
                      INNER JOIN {table_name2} AS {table_name2}                       
                      ON {table_name1}.mov_id = {table_name2}.mov_id      
                      INNER JOIN {table_name3} AS {table_name3}                       
                      ON {table_name2}.act_id = {table_name3}.act_id
                      INNER JOIN {table_name4} AS {table_name4}                       
                      ON {table_name1}.mov_id = {table_name4}.mov_id
                      INNER JOIN {table_name5} AS {table_name5}                       
                      ON {table_name4}.dir_id = {table_name5}.dir_id
                      WHERE {table_name3}.act_fname = "Claire" AND 
                            {table_name3}.act_lname = "Danes"  AND
                            {table_name3}.act_gender = "F"
                 ''')
pd.DataFrame(rs.fetchall(), columns = ['dir_fname', 'dir_lname', 'mov_title', 'role'])

,dir_fname,dir_lname,mov_title,role
0,Hayao,Miyazaki,PrincessMononoke,San


> * **Pandas**

In [84]:
table1, table2, table3 = db[table_name1], db[table_name2], db[table_name3]
table4, table5 = db[table_name4], db[table_name5]

SELECT_CLAUSE = ['dir_fname', 'dir_lname', 'mov_title', 'role']

WHERE_CONDITION1 = (table3['act_fname'] == "Claire")
WHERE_CONDITION2 = (table3['act_lname'] == "Danes")
WHERE_CONDITION3 = (table3['act_gender'] == "F")
WHERE_CLAUSE = WHERE_CONDITION1 & WHERE_CONDITION2 & WHERE_CONDITION3

rs = table1.merge(table2, 'inner', 'mov_id') \
           .merge(table3[WHERE_CLAUSE], 'inner','act_id') \
           .merge(table4, 'inner','mov_id') \
           .merge(table5, 'inner','dir_id') \
           [SELECT_CLAUSE]
rs

,dir_fname,dir_lname,mov_title,role
0,Hayao,Miyazaki,PrincessMononoke,San


### Query 46: Write a query in SQL to find the first and last name of an actor with their role in the movie which was also directed by themselve.

In [0]:
table_name1, table_name2, table_name3 = 'movie',  'movie_cast', 'actor',
table_name4, table_name5 = 'movie_direction', 'director'

> * **SQL**

In [86]:
rs = con.execute(f''' SELECT {table_name3}.act_fname, {table_name3}.act_lname,
                             {table_name1}.mov_title, {table_name2}.role
                      FROM {table_name1} AS {table_name1} 
                      INNER JOIN {table_name2} AS {table_name2}                       
                      ON {table_name1}.mov_id = {table_name2}.mov_id      
                      INNER JOIN {table_name3} AS {table_name3}                       
                      ON {table_name2}.act_id = {table_name3}.act_id
                      INNER JOIN {table_name4} AS {table_name4}                       
                      ON {table_name1}.mov_id = {table_name4}.mov_id
                      INNER JOIN {table_name5} AS {table_name5}                       
                      ON {table_name4}.dir_id = {table_name5}.dir_id
                      WHERE {table_name3}.act_fname = {table_name5}.dir_fname AND 
                            {table_name3}.act_lname = {table_name5}.dir_lname                            
                 ''')
pd.DataFrame(rs.fetchall(), columns = ['dir_fname', 'dir_lname', 'mov_title', 'role'])

,dir_fname,dir_lname,mov_title,role
0,Woody,Allen,AnnieHall,AlvySinger
1,Kevin,Spacey,BeyondtheSea,BobbyDarin


> * **Pandas**

In [87]:
table1, table2, table3 = db[table_name1], db[table_name2], db[table_name3]
table4, table5 = db[table_name4], db[table_name5]

SELECT_CLAUSE = ['dir_fname', 'dir_lname', 'mov_title', 'role']

rs = table1.merge(table2, 'inner', 'mov_id') \
           .merge(table3, 'inner','act_id') \
           .merge(table4, 'inner','mov_id') \
           .merge(table5, 'inner','dir_id')          

WHERE_CONDITION1 = (rs['act_fname'] == rs['dir_fname'])
WHERE_CONDITION2 = (rs['act_lname'] == rs['dir_lname'])
WHERE_CLAUSE = WHERE_CONDITION1 & WHERE_CONDITION2
           
rs = rs[WHERE_CLAUSE][SELECT_CLAUSE]
rs

,dir_fname,dir_lname,mov_title,role
10,Woody,Allen,AnnieHall,AlvySinger
14,Kevin,Spacey,BeyondtheSea,BobbyDarin


### Query 47: Write a query to find the cast list for the movie Chinatown.  

In [0]:
table_name1, table_name2, table_name3 = 'movie',  'movie_cast', 'actor',

> * **SQL**

In [89]:
rs = con.execute(f''' SELECT {table_name3}.act_fname, {table_name3}.act_lname
                      FROM {table_name1} AS {table_name1} 
                      INNER JOIN {table_name2} AS {table_name2}                       
                      ON {table_name1}.mov_id = {table_name2}.mov_id      
                      INNER JOIN {table_name3} AS {table_name3}                       
                      ON {table_name2}.act_id = {table_name3}.act_id                   
                      WHERE {table_name1}.mov_title = 'Chinatown'                            
                 ''')
pd.DataFrame(rs.fetchall(), columns = ['act_fname', 'act_lname'])

,act_fname,act_lname
0,Jack,Nicholson


> * **Pandas**

In [90]:
table1, table2, table3 = db[table_name1], db[table_name2], db[table_name3]

SELECT_CLAUSE = ['act_fname', 'act_lname']

WHERE_CONDITION = (table1['mov_title'] == 'Chinatown')
WHERE_CLAUSE = WHERE_CONDITION

rs = table1[WHERE_CLAUSE].merge(table2, 'inner', 'mov_id') \
                         .merge(table3, 'inner','act_id') \
                         [SELECT_CLAUSE]
rs

,act_fname,act_lname
0,Jack,Nicholson


### Query 48: Write a query to find the movie in which the actor appeared whose first and last name are 'Harrison' and 'Ford'. 

In [0]:
table_name1, table_name2, table_name3 = 'movie',  'movie_cast', 'actor',

> * **SQL**

In [92]:
rs = con.execute(f''' SELECT {table_name1}.mov_title
                      FROM {table_name1} AS {table_name1} 
                      INNER JOIN {table_name2} AS {table_name2}                       
                      ON {table_name1}.mov_id = {table_name2}.mov_id      
                      INNER JOIN {table_name3} AS {table_name3}                       
                      ON {table_name2}.act_id = {table_name3}.act_id                   
                      WHERE {table_name3}.act_fname = 'Harrison' AND
                            {table_name3}.act_lname = 'Ford'
                 ''')
pd.DataFrame(rs.fetchall(), columns = ['mov_title'])

,mov_title
0,BladeRunner


> * **Pandas**

In [93]:
table1, table2, table3 = db[table_name1], db[table_name2], db[table_name3]

SELECT_CLAUSE = ['mov_title']

WHERE_CONDITION1 = (table3['act_fname'] == 'Harrison')
WHERE_CONDITION2= (table3['act_lname'] == 'Ford')
WHERE_CLAUSE = WHERE_CONDITION1 & WHERE_CONDITION2

rs = table1.merge(table2, 'inner', 'mov_id') \
           .merge(table3[WHERE_CLAUSE], 'inner','act_id') \
           [SELECT_CLAUSE]
rs

,mov_title
0,BladeRunner


### Query 49: Write a query to find the highest-rated movie, and report its title, year, rating, and releasing country.  

In [0]:
table_name1, table_name2 = 'movie',  'rating'

> * **SQL**

In [95]:
rs = con.execute(f''' SELECT {table_name1}.mov_title, {table_name1}.mov_year, 
                             {table_name2}.num_of_ratings, {table_name1}.mov_rel_country
                      FROM {table_name1} AS {table_name1} 
                      INNER JOIN {table_name2} AS {table_name2}                       
                      ON {table_name1}.mov_id = {table_name2}.mov_id                          
                      WHERE {table_name2}.num_of_ratings = (SELECT MAX(num_of_ratings) FROM {table_name2})                            
                 ''')
pd.DataFrame(rs.fetchall(), columns = ['mov_title', 'mov_year', 'num_of_ratings', 'mov_rel_country'])

,mov_title,mov_year,num_of_ratings,mov_rel_country
0,AmericanBeauty,1999,862618.0,UK


> * **Pandas**

In [96]:
table1, table2 = db[table_name1], db[table_name2]

SELECT_CLAUSE = ['mov_title', 'mov_year', 'num_of_ratings', 'mov_rel_country']


WHERE_CONDITION1= (table2['num_of_ratings'] == table2['num_of_ratings'].max())
WHERE_CLAUSE = WHERE_CONDITION1

rs = table1.merge(table2[WHERE_CLAUSE], 'inner', 'mov_id') \
           [SELECT_CLAUSE]
rs

,mov_title,mov_year,num_of_ratings,mov_rel_country
0,AmericanBeauty,1999,862618.0,UK


### Query 50: Write a query to find the highest-rated Mystery movie, and report the title, year, and rating. 

In [0]:
table_name1, table_name2 = 'movie', 'rating'
table_name3, table_name4 = 'movie_genres', 'genres'

> * **SQL**

In [98]:
rs = con.execute(f''' SELECT {table_name1}.mov_title, {table_name1}.mov_year, 
                             {table_name2}.num_of_ratings
                      FROM {table_name1} AS {table_name1}                       
                      INNER JOIN {table_name2} AS {table_name2}                       
                      ON {table_name1}.mov_id = {table_name2}.mov_id                          
                      INNER JOIN {table_name3} AS {table_name3}                       
                      ON {table_name1}.mov_id = {table_name3}.mov_id                          
                      INNER JOIN {table_name4} AS {table_name4}                       
                      ON {table_name3}.gen_id = {table_name4}.gen_id                          
                      WHERE {table_name4}.gen_title = "Mystery"                           
                 ''')
pd.DataFrame(rs.fetchall(), columns = ['mov_title', 'mov_year', 'num_of_ratings'])

,mov_title,mov_year,num_of_ratings
0,Vertigo,1958,263575.0


> * **Pandas**

In [99]:
table1, table2 = db[table_name1], db[table_name2]
table3, table4 = db[table_name3], db[table_name4]

SELECT_CLAUSE = ['mov_title', 'mov_year', 'num_of_ratings']

WHERE_CONDITION= (table4['gen_title'] == 'Mystery')
WHERE_CLAUSE = WHERE_CONDITION

rs = table1.merge(table2, 'inner', 'mov_id') \
           .merge(table3, 'inner', 'mov_id') \
           .merge(table4[WHERE_CLAUSE], 'inner', 'gen_id') \
           [SELECT_CLAUSE]
rs

,mov_title,mov_year,num_of_ratings
0,Vertigo,1958,263575.0


### Query 51:  Write a query to generate a report which shows the year when most of the Mystery movies produces, and number of movies and their average rating. 

In [0]:
table_name1, table_name2 = 'movie', 'rating'
table_name3, table_name4 = 'movie_genres', 'genres'

> * **SQL**

In [101]:
rs = con.execute(f''' SELECT {table_name1}.mov_year, COUNT({table_name1}.mov_id), AVG({table_name2}.num_of_ratings)                              
                      FROM {table_name1} AS {table_name1}                       
                      LEFT JOIN {table_name2} AS {table_name2}                       
                      ON {table_name1}.mov_id = {table_name2}.mov_id                          
                      INNER JOIN {table_name3} AS {table_name3}                       
                      ON {table_name1}.mov_id = {table_name3}.mov_id                          
                      INNER JOIN {table_name4} AS {table_name4}                       
                      ON {table_name3}.gen_id = {table_name4}.gen_id                          
                      WHERE {table_name4}.gen_title = "Mystery"  
                      GROUP BY {table_name1}.mov_year                      
                 ''')
pd.DataFrame(rs.fetchall(), columns = ['mov_year', 'num_of_mystery_mov', 'avg_ratings'])

,mov_year,num_of_mystery_mov,avg_ratings
0,1958,1,263575.0
1,1999,1,NaN
2,2001,1,NaN


> * **Pandas**

In [102]:
table1, table2 = db[table_name1], db[table_name2]
table3, table4 = db[table_name3], db[table_name4]

SELECT_CLAUSE = ['mov_title']

WHERE_CONDITION= (table4['gen_title'] == 'Mystery')
WHERE_CLAUSE = WHERE_CONDITION
AGG_OP = {'mov_id': 'count', 'num_of_ratings': 'mean'}
rs = table1.merge(table2, 'left', 'mov_id') \
           .merge(table3, 'inner', 'mov_id') \
           .merge(table4[WHERE_CLAUSE], 'inner', 'gen_id') \
           .groupby(['mov_year']) \
           .agg(AGG_OP) \
           
rs

,mov_id,num_of_ratings
mov_year,,
1958,1,263575.0
1999,1,NaN
2001,1,NaN


### Query 52: Write a query to generate a report which contain the columns movie title, name of the female actor, year of the movie, role, movie genres, the director, date of release, and rating of that movie. 

In [0]:
table_name1, table_name2, table_name3 = 'movie', 'movie_direction', 'director'
table_name4, table_name5, table_name6 = 'movie_cast', 'actor', 'movie_genres', 
table_name7, table_name8 = 'genres', 'rating'

> * **SQL**

In [104]:
rs = con.execute(f''' SELECT {table_name1}.mov_title, {table_name5}.act_fname, {table_name5}.act_lname,
                             {table_name1}.mov_year, {table_name4}.role, {table_name1}.mov_dt_rel,
                             {table_name8}.num_of_ratings
                      FROM {table_name1} AS {table_name1}                       
                      INNER JOIN {table_name2} AS {table_name2}                       
                      ON {table_name1}.mov_id = {table_name2}.mov_id                          
                      INNER JOIN {table_name3} AS {table_name3}                       
                      ON {table_name2}.dir_id = {table_name3}.dir_id 
                      INNER JOIN {table_name4} AS {table_name4}                       
                      ON {table_name1}.mov_id = {table_name4}.mov_id
                      INNER JOIN {table_name5} AS {table_name5}                       
                      ON {table_name4}.act_id = {table_name5}.act_id                  
                      INNER JOIN {table_name6} AS {table_name6}                       
                      ON {table_name1}.mov_id = {table_name6}.mov_id                  
                      INNER JOIN {table_name7} AS {table_name7}                       
                      ON {table_name6}.gen_id = {table_name7}.gen_id                          
                      INNER JOIN {table_name8} AS {table_name8}                       
                      ON {table_name1}.mov_id = {table_name8}.mov_id                          
                      WHERE {table_name5}.act_gender = "F"                                          
                 ''')
pd.DataFrame(rs.fetchall(), columns = ['mov_title', 'act_fname', 'act_lname', 
                                       'mov_year', 'role', 'mov_dt_rel',
                                       'num_of_ratings'])

,mov_title,act_fname,act_lname,mov_year,role,mov_dt_rel,num_of_ratings
0,TheInnocents,Deborah,Kerr,1961,MissGiddens,1962-02-19,20207.0
1,PrincessMononoke,Claire,Danes,1997,San,2001-10-19,NaN
2,Aliens,Sigourney,Weaver,1986,Ripley,1986-08-29,511613.0


> * **Pandas**

In [105]:
table1, table2, table3 = db[table_name1], db[table_name2], db[table_name3]
table4, table5, table6 = db[table_name4], db[table_name5], db[table_name6]
table7, table8 = db[table_name7], db[table_name8]

SELECT_CLAUSE = ['mov_title', 'act_fname', 'act_lname', 'mov_year', 
                 'role', 'mov_dt_rel', 'num_of_ratings']

WHERE_CONDITION= (table5['act_gender'] == 'F')
WHERE_CLAUSE = WHERE_CONDITION

rs = table1.merge(table2, 'inner', 'mov_id') \
           .merge(table3, 'inner', 'dir_id') \
           .merge(table4, 'inner', 'mov_id') \
           .merge(table5[WHERE_CLAUSE], 'inner', 'act_id') \
           .merge(table6, 'inner', 'mov_id') \
           .merge(table7, 'inner', 'gen_id') \
           .merge(table8, 'inner', 'mov_id') \
           [SELECT_CLAUSE]          
rs

,mov_title,act_fname,act_lname,mov_year,role,mov_dt_rel,num_of_ratings
0,TheInnocents,Deborah,Kerr,1961,MissGiddens,1962-02-19,20207.0
1,PrincessMononoke,Claire,Danes,1997,San,2001-10-19,NaN
2,Aliens,Sigourney,Weaver,1986,Ripley,1986-08-29,511613.0
